import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
import pandas as pd
from IPython.display import display
import sys

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min, col, rank
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [2]:
import findspark
findspark.init()

spark = SparkSession.builder \
    .appName("SparkTask") \
    .master("local") \
    .getOrCreate()

read csv with inferschema

In [3]:
csv_path = "./ds_salaries.csv"

df = spark.read.csv(csv_path, header=True, inferSchema=True)

df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [3]:
csv_path = "./ds_salaries.csv"

df = spark.read.csv(csv_path, header=True, inferSchema=True)

df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

write schema of scv on screen

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [3]:
# sample_df = spark.read.option("header", "true").option("inferSchema", "true").csv(csv_path)

# sample_df.printSchema()

custom_schema = StructType([
    StructField("_c0", IntegerType(), True),
    StructField("work_year", IntegerType(), True),
    StructField("experience_level", StringType(), True),
    StructField("employment_type", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("salary_currency", StringType(), True),
    StructField("salary_in_usd", IntegerType(), True),
    StructField("employee_residence", StringType(), True),
    StructField("remote_ratio", IntegerType(), True),
    StructField("company_location", StringType(), True),
    StructField("company_size", StringType(), True)
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
csv_path = "./ds_salaries.csv"

df = spark.read.csv(csv_path, header=True, schema=custom_schema)

df.show()

NameError: name 'custom_schema' is not defined

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [4]:
pandas_df = df.toPandas()
print(pandas_df)

# so basically there is no disutils tool in python 3.12.0

# df_pd = df.limit(10).select("*").toPandas()

Py4JError: An error occurred while calling o33.pandasStructHandlingMode. Trace:
py4j.Py4JException: Method pandasStructHandlingMode([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:842)



create df_job_title that consists from all job_titles without duplicates

In [4]:
selected_column = "job_title"

df_job_title = df.select(col(selected_column)).distinct()

df_job_title.show()

+--------------------+
|           job_title|
+--------------------+
|3D Computer Visio...|
|  Lead Data Engineer|
|Head of Machine L...|
|     Data Specialist|
| Data Analytics Lead|
|Machine Learning ...|
|   Lead Data Analyst|
|Data Engineering ...|
|Staff Data Scientist|
|       ETL Developer|
|Director of Data ...|
|Product Data Analyst|
|Principal Data Sc...|
|        AI Scientist|
|Director of Data ...|
|Machine Learning ...|
| Lead Data Scientist|
|Machine Learning ...|
|Data Science Engi...|
|Machine Learning ...|
+--------------------+
only showing top 20 rows



print all rows from df_job_titles without truncating jobs

In [5]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [5]:
df_analytic = (
    df.groupBy("job_title")
        .agg(avg("salary_in_usd").alias("avg_salary"),
           min("salary_in_usd").alias("min_salary"),
           max("salary_in_usd").alias("max_salary"))
)

df_analytic.show()

+--------------------+------------------+----------+----------+
|           job_title|        avg_salary|min_salary|max_salary|
+--------------------+------------------+----------+----------+
|3D Computer Visio...|            5409.0|      5409|      5409|
|  Lead Data Engineer|          139724.5|     56000|    276000|
|Head of Machine L...|           79039.0|     79039|     79039|
|     Data Specialist|          165000.0|    165000|    165000|
| Data Analytics Lead|          405000.0|    405000|    405000|
|Machine Learning ...|          158412.5|     12000|    260000|
|   Lead Data Analyst|           92203.0|     19609|    170000|
|Data Engineering ...|          123227.2|     59303|    174000|
|Staff Data Scientist|          105000.0|    105000|    105000|
|       ETL Developer|           54957.0|     54957|     54957|
|Director of Data ...|          156738.0|    113476|    200000|
|Product Data Analyst|           13036.0|      6072|     20000|
|Principal Data Sc...|215242.42857142858

print all rows from df_analytic without trancating jobs

In [8]:
df_analytic.show(truncate=False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409      |5409      |
|Lead Data Engineer                      |139724.5          |56000     |276000    |
|Head of Machine Learning                |79039.0           |79039     |79039     |
|Data Specialist                         |165000.0          |165000    |165000    |
|Data Analytics Lead                     |405000.0          |405000    |405000    |
|Machine Learning Scientist              |158412.5          |12000     |260000    |
|Lead Data Analyst                       |92203.0           |19609     |170000    |
|Data Engineering Manager                |123227.2          |59303     |174000    |
|Staff Data Scientist                    |105000.0          |105000    |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [6]:
windowSpec = Window.orderBy(col("avg_salary").desc())

df_analytic = (
    df.groupBy("job_title")
        .agg(avg("salary_in_usd").alias("avg_salary"),
           min("salary_in_usd").alias("min_salary"),
           max("salary_in_usd").alias("max_salary"))
        .withColumn("row_id", rank().over(windowSpec))
)

df_analytic.show()

+--------------------+------------------+----------+----------+------+
|           job_title|        avg_salary|min_salary|max_salary|row_id|
+--------------------+------------------+----------+----------+------+
| Data Analytics Lead|          405000.0|    405000|    405000|     1|
|Principal Data En...| 328333.3333333333|    185000|    600000|     2|
|Financial Data An...|          275000.0|    100000|    450000|     3|
|Principal Data Sc...|215242.42857142858|    148261|    416000|     4|
|Director of Data ...|          195074.0|    130026|    325000|     5|
|      Data Architect| 177873.9090909091|     90700|    266400|     6|
|Applied Data Scie...|          175655.0|     54238|    380000|     7|
|  Analytics Engineer|          175000.0|    135000|    205300|     8|
|     Data Specialist|          165000.0|    165000|    165000|     9|
|        Head of Data|          160162.6|     32974|    235000|    10|
|Machine Learning ...|          158412.5|     12000|    260000|    11|
|Data 

print all data from df_analytic

In [13]:
df_analytic.show()

+--------------------+------------------+----------+----------+------+
|           job_title|        avg_salary|min_salary|max_salary|row_id|
+--------------------+------------------+----------+----------+------+
| Data Analytics Lead|          405000.0|    405000|    405000|     1|
|Principal Data En...| 328333.3333333333|    185000|    600000|     2|
|Financial Data An...|          275000.0|    100000|    450000|     3|
|Principal Data Sc...|215242.42857142858|    148261|    416000|     4|
|Director of Data ...|          195074.0|    130026|    325000|     5|
|      Data Architect| 177873.9090909091|     90700|    266400|     6|
|Applied Data Scie...|          175655.0|     54238|    380000|     7|
|  Analytics Engineer|          175000.0|    135000|    205300|     8|
|     Data Specialist|          165000.0|    165000|    165000|     9|
|        Head of Data|          160162.6|     32974|    235000|    10|
|Machine Learning ...|          158412.5|     12000|    260000|    11|
|Data 

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [7]:
new_order = ["row_id"] + [col for col in df_analytic.columns if col != "row_id"]

df_analytic_reordered = df_analytic.select(*new_order)

df_analytic_reordered.show()

+------+--------------------+------------------+----------+----------+
|row_id|           job_title|        avg_salary|min_salary|max_salary|
+------+--------------------+------------------+----------+----------+
|     1| Data Analytics Lead|          405000.0|    405000|    405000|
|     2|Principal Data En...| 328333.3333333333|    185000|    600000|
|     3|Financial Data An...|          275000.0|    100000|    450000|
|     4|Principal Data Sc...|215242.42857142858|    148261|    416000|
|     5|Director of Data ...|          195074.0|    130026|    325000|
|     6|      Data Architect| 177873.9090909091|     90700|    266400|
|     7|Applied Data Scie...|          175655.0|     54238|    380000|
|     8|  Analytics Engineer|          175000.0|    135000|    205300|
|     9|     Data Specialist|          165000.0|    165000|    165000|
|    10|        Head of Data|          160162.6|     32974|    235000|
|    11|Machine Learning ...|          158412.5|     12000|    260000|
|    1

print df_analytic now

In [15]:
df_analytic_reordered.show()

+------+--------------------+------------------+----------+----------+
|row_id|           job_title|        avg_salary|min_salary|max_salary|
+------+--------------------+------------------+----------+----------+
|     1| Data Analytics Lead|          405000.0|    405000|    405000|
|     2|Principal Data En...| 328333.3333333333|    185000|    600000|
|     3|Financial Data An...|          275000.0|    100000|    450000|
|     4|Principal Data Sc...|215242.42857142858|    148261|    416000|
|     5|Director of Data ...|          195074.0|    130026|    325000|
|     6|      Data Architect| 177873.9090909091|     90700|    266400|
|     7|Applied Data Scie...|          175655.0|     54238|    380000|
|     8|  Analytics Engineer|          175000.0|    135000|    205300|
|     9|     Data Specialist|          165000.0|    165000|    165000|
|    10|        Head of Data|          160162.6|     32974|    235000|
|    11|Machine Learning ...|          158412.5|     12000|    260000|
|    1

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [8]:
windowSpec = Window().partitionBy("experience_level").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_exp_lvl = (
    df.withColumn("biggest_salary", max("salary_in_usd").over(windowSpec))
      .groupBy("experience_level")
      .agg(max("salary_in_usd").alias("biggest_salary"))
)

df_exp_lvl.show()

+----------------+--------------+
|experience_level|biggest_salary|
+----------------+--------------+
|              EX|        600000|
|              MI|        450000|
|              EN|        250000|
|              SE|        412000|
+----------------+--------------+



In [14]:
filtered_df = df.where((col("salary_in_usd") == 600000) & (col("experience_level") == "EX") | 
                       (col("salary_in_usd") == 450000) & (col("experience_level") == "MI") | 
                       (col("salary_in_usd") == 2500000) & (col("experience_level") == "EN") | 
                       (col("salary_in_usd") == 412000) & (col("experience_level") == "SE"))

filtered_df.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
| 63|     2020|              SE|             FT|      Data Scientist|412000|            USD|       412000|                US|         100|              US|           L|
| 97|     2021|              MI|             FT|Financial Data An...|450000|            USD|       450000|                US|         100|              US|

print here df_exp_lvl

In [15]:
filtered_df.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
| 63|     2020|              SE|             FT|      Data Scientist|412000|            USD|       412000|                US|         100|              US|           L|
| 97|     2021|              MI|             FT|Financial Data An...|450000|            USD|       450000|                US|         100|              US|

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [18]:
df_best = filtered_df.select(col("_c0").alias("id"), col("experience_level"), col("salary_in_usd").alias("biggest_salary"), col("employee_residence"))

df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



print df_best

drop duplicates if exist by experience_level

print df_best

create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

print df_new_best

write df_new_best like 1.csv and load then it to df_final

print df_final

filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

print df_final

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system